# Cython Optimize Zeros API
The Cython Optimize Zeros API let's users interface with the root finders for scalar functions in the SciPy Optimize package. In order to use it, users must write and compile Cython code. There are several ways to do that, which are described in the Cython Documentation.

## Cython Jupyter Extension
The Cython Jupyter extension is one method of compiling Cython code that is described in the Cython Documentation. To enable it first use the following magic:

    %load_ext Cython

In [1]:
%load_ext cython

## Compiling Cython in Jupyter
To compile Cython in Jupyter start with `%%cython` magic in the cell you want to compile. The following test is copied verbatim from the Cython Documentation

In [2]:
%%cython

cdef int a = 0
for i in range(10):
    a += i
print(a)

45


## SciPy Cython Special API
The Scipy Optimize Zeros API was inspired by the Cython Special API. The following test is from the SciPy Specials documentation.

In [3]:
%%cython

cimport scipy.special.cython_special as csc

cdef:
    double x = 1
    double complex z = 1 + 1j
    double si, ci, rgam
    double complex cgam

rgam = csc.gamma(x)
print(rgam)
cgam = csc.gamma(z)
print(cgam)
csc.sici(x, &si, &ci)
print(si, ci)


1.0
(0.49801566811835557-0.15494982830181037j)
0.9460830703671831 0.33740392290096816


## Example Usage of Cython Optimize Zeros API
The following example using the `double (*callback_type_tuple)(double x, tuple args)` callback signature with the `brentq` root-finder to find the root of the function $f(x) = 1 - \exp(-(x - 0.7))$.

1. first import the `cython_optimize` package using `cimport`
2. then write the callback in Cython with the matching signature
3. finally call the root-finder with its expected signature

In [4]:
%%cython

import math
cimport scipy.optimize.cython_optimize as coz

cdef double f(double x, tuple args):
    return 1.0 - math.exp(-(x - 0.7))


print(coz.zeros_tuple.brentq(f, 0.5, 1.0, (), 1e-3, 1e-3, 10, NULL))

0.6999942848231314


### Validation
Validate the result with the existing method.

In [5]:
from scipy.optimize import brentq

brentq(lambda x: 1.0 - math.exp(-(x - 0.7)), 0.5, 1.0, (), 1e-3, 1e-3, 10)

0.6999942848231314

### `struct` Signature
Try the signature `double (*callback_type)(double x, void *args)`. This callback takes a double and a `struct` we call `test_params` that in this example has extra parameters, `C0` and `C1`, that are used in the function. The `brentq` root-finder corresponding to this callback is in `scipy.optimize.cython_optimize.zeros_struct`.

In [6]:
%%cython --annotate

import math
cimport scipy.optimize.cython_optimize as coz

ctypedef struct test_params:
    double C0
    double C1
    

cdef double f(double x, void *args):
    cdef test_params *myargs = <test_params *> args
    return myargs.C0 - math.exp(-(x - myargs.C1))


cdef test_params myargs
myargs.C0 = 1.0
myargs.C1 = 0.7

print(coz.zeros_struct.brentq(f, 0.5, 1.0, <test_params *> &myargs, 1e-3, 1e-3, 10, NULL))

0.6999942848231314


### Array Signature
The last signature `double (*callback_type_array)(int n, double *args)` takes an array and its length as inputs. The solvers for this signature are in `scipy.optimize.cython_optimize.zeros_array`. When writing the callback for this signature, the length of the array, `n`, is probably not used directly, but it must still be passed to the solver along with the array of extra arguments. The independent variable should be prepended to the beginning of the array when it is used in the callback, but **not** when it is passed to the solver.

In [7]:
%%cython --annotate

import math
cimport cpython
cimport scipy.optimize.cython_optimize as coz

ctypedef struct test_params:
    double constant

DEF N = 2  # define a compile time constant using "DEF"

cdef double f(int n, double* args):
    # the first "arg" must be the independent variable "x"
    return args[1] - math.exp(-(args[0] - args[2]))


cdef int n = N  # the number of "extra arguments"
# use the compile time constant "N" to allocate the array
cdef double[N] myargs = (1.0, 0.7)  # extra arguments

print(coz.zeros_array.brentq(f, 0.5, 1, n, myargs, 1e-3, 1e-3, 10, NULL))

0.6999942848231314


## Full Ouptut
If a C `struct` is passed to the solver as its final argument, then full output with the fields `funcalls`, `iterations`, and `error_num` will be copied to the C `struct`. Otherwise, this argument should be passed as `NULL`. The four C functions from `scipy/optmize/Zeros` use `scipy_zeros_parameters` while the three C functions from `scipy/optimize/cython_optmize/Newton` use `scipy_newton_parameters`. You may overload the C `struct` to add or remove any additional fields you choose.

In [8]:
%%cython
from scipy.optimize.cython_optimize cimport zeros_tuple
from scipy.optimize.cython_optimize.examples.zeros_tuple_examples cimport f_solarcell, fprime, fprime2

ARGS = (5.25, 6.0, 1e-09, 0.004, 10.0, 0.27456)
TOL, MAXITER = 1.48e-8, 50


ctypedef struct scipy_newton_full_output:
    int funcalls
    int iterations
    int error_num
    double root


# cython newton solver with full output
cdef scipy_newton_full_output solarcell_newton_full_output(tuple args, double x0, double tol, int maxiter):
    cdef scipy_newton_full_output full_output
    full_output.root = zeros_tuple.newton(f_solarcell, x0, fprime, args, tol, maxiter, <zeros_tuple.scipy_newton_parameters *> &full_output)
    return full_output


# cython secant solver with full output
cdef scipy_newton_full_output solarcell_secant_full_output(tuple args, double x0, double tol, int maxiter):
    cdef scipy_newton_full_output full_output
    full_output.root = zeros_tuple.secant(f_solarcell, x0, args, tol, maxiter, <zeros_tuple.scipy_newton_parameters *> &full_output)
    return full_output


# cython halley solver with full output
cdef scipy_newton_full_output solarcell_halley_full_output(tuple args, double x0, double tol, int maxiter):
    cdef scipy_newton_full_output full_output
    full_output.root = zeros_tuple.halley(f_solarcell, x0, fprime, args, tol, maxiter, fprime2, <zeros_tuple.scipy_newton_parameters *> &full_output)
    return full_output


def test_newton_full_output(args=ARGS, x0=6.0, tol=TOL, maxiter=MAXITER):
    """test newton with full output"""
    return {'newton': solarcell_newton_full_output(args, x0, tol, maxiter),
            'secant': solarcell_secant_full_output(args, x0, tol, maxiter),
            'halley': solarcell_halley_full_output(args, x0, tol, maxiter)}

In [9]:
test_newton_full_output()

{'newton': {'funcalls': 6,
  'iterations': 3,
  'error_num': 0,
  'root': 5.255320079106907},
 'secant': {'funcalls': 4,
  'iterations': 3,
  'error_num': 0,
  'root': 5.255320079106908},
 'halley': {'funcalls': 7,
  'iterations': 3,
  'error_num': 0,
  'root': 5.255320079106907}}

### Full Output with Brent Solvers and Tuple Signature
The solvers in `scipy/optimize/Zeros` use `scipy_zeros_parameters` for full output.

In [10]:
%%cython
from scipy.optimize.cython_optimize cimport zeros_tuple
from scipy.optimize.cython_optimize.examples.zeros_tuple_examples cimport f_solarcell, fprime, fprime2

ARGS = (5.25, 6.0, 1e-09, 0.004, 10.0, 0.27456)
XTOL, RTOL, MITR = 0.001, 0.001, 10

ctypedef struct scipy_brent_full_output:
    int funcalls
    int iterations
    int error_num
    double root

# cython brentq solver with full output
cdef scipy_brent_full_output solarcell_brentq_full_output(tuple args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_tuple.brentq(f_solarcell, xa, xb, args, xtol, rtol, mitr, <zeros_tuple.scipy_zeros_parameters *> &full_output)
    return full_output

# cython brenth solver with full output
cdef scipy_brent_full_output solarcell_brenth_full_output(tuple args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_tuple.brenth(f_solarcell, xa, xb, args, xtol, rtol, mitr, <zeros_tuple.scipy_zeros_parameters *> &full_output)
    return full_output

# cython ridder solver with full output
cdef scipy_brent_full_output solarcell_ridder_full_output(tuple args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_tuple.ridder(f_solarcell, xa, xb, args, xtol, rtol, mitr, <zeros_tuple.scipy_zeros_parameters *> &full_output)
    return full_output

# cython bisect solver with full output
cdef scipy_brent_full_output solarcell_bisect_full_output(tuple args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_tuple.bisect(f_solarcell, xa, xb, args, xtol, rtol, mitr, <zeros_tuple.scipy_zeros_parameters *> &full_output)
    return full_output


def test_brent_full_output(args=ARGS, xa=0.0, xb=6.0, xtol=XTOL, rtol=RTOL, mitr=MITR):
    """test brent with full output"""
    return {'brentq': solarcell_brentq_full_output(args, xa, xb, xtol, rtol, mitr),
            'brenth': solarcell_brenth_full_output(args, xa, xb, xtol, rtol, mitr),
            'ridder': solarcell_ridder_full_output(args, xa, xb, xtol, rtol, mitr),
            'bisect': solarcell_bisect_full_output(args, xa, xb, xtol, rtol, mitr)}

In [11]:
test_brent_full_output()

{'brentq': {'funcalls': 4,
  'iterations': 3,
  'error_num': 0,
  'root': 5.255231961257658},
 'brenth': {'funcalls': 4,
  'iterations': 3,
  'error_num': 0,
  'root': 5.255231961257658},
 'ridder': {'funcalls': 6,
  'iterations': 2,
  'error_num': 0,
  'root': 5.258446778558067},
 'bisect': {'funcalls': 12,
  'iterations': 10,
  'error_num': 0,
  'root': 5.255859375}}

#### Validation
Get the full output from ``scipy.optimize.newton`` to compare. LGTM!

In [12]:
from math import exp
from scipy.optimize import newton

ARGS = (5.25, 6.0, 1e-09, 0.004, 10.0, 0.27456)
TOL, MAXITER = 1.48e-8, 50


def f_solarcell(i, *args):
    v, il, io, rs, rsh, vt = args
    vd = v + i * rs
    return il - io * (exp(vd / vt) - 1.0) - vd / rsh - i


newton(f_solarcell, 6.0, fprime=None, args=ARGS, tol=TOL, maxiter=MAXITER, fprime2=None, full_output=True)

(5.255320079106908,       converged: True
            flag: 'converged'
  function_calls: 4
      iterations: 3
            root: 5.255320079106908)

In [13]:
def fprime(i, *args):
    v, il, io, rs, rsh, vt = args
    return -io * exp((v + i * rs) / vt) * rs / vt - rs / rsh - 1

newton(f_solarcell, 6.0, fprime=fprime, args=ARGS, tol=TOL, maxiter=MAXITER, fprime2=None, full_output=True)

(5.255320079106907,       converged: True
            flag: 'converged'
  function_calls: 6
      iterations: 3
            root: 5.255320079106907)

In [14]:
def fprime2(i, *args):
    v, il, io, rs, rsh, vt = args
    return -io * exp((v + i * rs) / vt) * (rs / vt)**2

newton(f_solarcell, 6.0, fprime=fprime, args=ARGS, tol=TOL, maxiter=MAXITER, fprime2=fprime2, full_output=True)

(5.255320079106907,       converged: True
            flag: 'converged'
  function_calls: 7
      iterations: 2
            root: 5.255320079106907)

In [15]:
from scipy.optimize import brentq, brenth, ridder, bisect

XTOL, RTOL, MITR = 0.001, 0.001, 10

brentq(f_solarcell, 0.0, 6.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(5.255231961257658,       converged: True
            flag: 'converged'
  function_calls: 4
      iterations: 3
            root: 5.255231961257658)

In [16]:
brenth(f_solarcell, 0.0, 6.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(5.255231961257658,       converged: True
            flag: 'converged'
  function_calls: 4
      iterations: 3
            root: 5.255231961257658)

In [17]:
ridder(f_solarcell, 0.0, 6.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(5.258446778558067,       converged: True
            flag: 'converged'
  function_calls: 6
      iterations: 2
            root: 5.258446778558067)

In [18]:
bisect(f_solarcell, 0.0, 6.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(5.255859375,       converged: True
            flag: 'converged'
  function_calls: 12
      iterations: 10
            root: 5.255859375)

### Full Output with Other Callbacks and Solvers
Let's try the full output from some of the other callbacks.

In [19]:
%%cython

import math
from scipy.optimize.cython_optimize cimport zeros_struct

ARGS = dict(C0=1.0, C1=0.7)
XTOL, RTOL, MITR = 0.001, 0.001, 10

ctypedef struct scipy_brent_full_output:
    int funcalls
    int iterations
    int error_num
    double root

ctypedef struct test_params:
    double C0
    double C1
    

cdef double f(double x, void *args):
    cdef test_params *myargs = <test_params *> args
    return myargs.C0 - math.exp(-(x - myargs.C1))


# cython brentq solver with full output
cdef scipy_brent_full_output show_brentq_full_output(dict args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef test_params myargs = args  # Cython automatically casts a dictionary to a C struct
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_struct.brentq(f, xa, xb, <test_params *> &myargs, xtol, rtol, mitr, <zeros_struct.scipy_zeros_parameters *> &full_output)
    return full_output


# cython brenth solver with full output
cdef scipy_brent_full_output show_brenth_full_output(dict args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef test_params myargs = args  # Cython automatically casts a dictionary to a C struct
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_struct.brenth(f, xa, xb, <test_params *> &myargs, xtol, rtol, mitr, <zeros_struct.scipy_zeros_parameters *> &full_output)
    return full_output


# cython ridder solver with full output
cdef scipy_brent_full_output show_ridder_full_output(dict args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef test_params myargs = args  # Cython automatically casts a dictionary to a C struct
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_struct.ridder(f, xa, xb, <test_params *> &myargs, xtol, rtol, mitr, <zeros_struct.scipy_zeros_parameters *> &full_output)
    return full_output


# cython bisect solver with full output
cdef scipy_brent_full_output show_bisect_full_output(dict args, double xa, double xb, double xtol, double rtol, int mitr):
    cdef test_params myargs = args  # Cython automatically casts a dictionary to a C struct
    cdef scipy_brent_full_output full_output
    full_output.root = zeros_struct.bisect(f, xa, xb, <test_params *> &myargs, xtol, rtol, mitr, <zeros_struct.scipy_zeros_parameters *> &full_output)
    return full_output


def test_brent_full_output(args=ARGS, xa=0.5, xb=1.0, xtol=XTOL, rtol=RTOL, mitr=MITR):
    """test newton with full output"""
    return {'brentq': show_brentq_full_output(args, xa, xb, xtol, rtol, mitr),
            'brenth': show_brenth_full_output(args, xa, xb, xtol, rtol, mitr),
            'ridder': show_ridder_full_output(args, xa, xb, xtol, rtol, mitr),
            'bisect': show_bisect_full_output(args, xa, xb, xtol, rtol, mitr)}

In [20]:
test_brent_full_output()

{'brentq': {'funcalls': 6,
  'iterations': 5,
  'error_num': 0,
  'root': 0.6999942848231314},
 'brenth': {'funcalls': 6,
  'iterations': 5,
  'error_num': 0,
  'root': 0.6999985100409557},
 'ridder': {'funcalls': 6,
  'iterations': 2,
  'error_num': 0,
  'root': 0.6992747140196313},
 'bisect': {'funcalls': 11,
  'iterations': 9,
  'error_num': 0,
  'root': 0.7001953125}}

In [21]:
from scipy.optimize import brentq

ARGS = (1.0, 0.7)
XTOL, RTOL, MITR = 0.001, 0.001, 10

brentq(lambda x, c0, c1: c0 - math.exp(-(x - c1)), 0.5, 1.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(0.6999942848231314,       converged: True
            flag: 'converged'
  function_calls: 6
      iterations: 5
            root: 0.6999942848231314)

In [22]:
brenth(lambda x, c0, c1: c0 - math.exp(-(x - c1)), 0.5, 1.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(0.6999985100409557,       converged: True
            flag: 'converged'
  function_calls: 6
      iterations: 5
            root: 0.6999985100409557)

In [23]:
ridder(lambda x, c0, c1: c0 - math.exp(-(x - c1)), 0.5, 1.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(0.6992747140196313,       converged: True
            flag: 'converged'
  function_calls: 6
      iterations: 2
            root: 0.6992747140196313)

In [24]:
bisect(lambda x, c0, c1: c0 - math.exp(-(x - c1)), 0.5, 1.0, ARGS, XTOL, RTOL, MITR, full_output=True)

(0.7001953125,       converged: True
            flag: 'converged'
  function_calls: 11
      iterations: 9
            root: 0.7001953125)

## More examples
It would be great to see more examples of how this low-level Cython API can be used with the scalar-function root-finders to improve performance. I'd especially like to see a tight loop that either uses NumPy arrays, Cython Memoryviews, or the Cython parallel `prange` function. Please feel free to add more examples either to this notebook or in the examples folder, and in either Jupyter noteooks or Python/Cython modules. Thanks!